# Exemplo de Classificação: **Classificação de imagens** de Cachorros e Gatos


---


- Exemplo adotado no curso **ABCIA**
- Código baseado em: https://www.tensorflow.org/tutorials/images/transfer_learning?hl=pt-br

In [ ]:
#@title Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet                                                                                                                    # IGNORE_COPYRIGHT: cleared by OSS licensing
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

In [ ]:
#@title Verificando GPU
# Exemplo: Testa T4 - 16GB
# NVIDIA CUDA® cores: 2,560
# https://www.nvidia.com/pt-br/data-center/tesla-t4/
--------------- TODO

# Transferência de aprendizado e ajuste fino

Neste tutorial, você aprenderá a classificar imagens de cães e gatos usando o aprendizado de transferência de uma rede pré-treinada.

Um modelo pré-treinado é uma rede salva que foi previamente treinada em um grande conjunto de dados, normalmente em uma tarefa de classificação de imagem em grande escala. Você usa o modelo pré-treinado como está ou usa o aprendizado de transferência para personalizar esse modelo para uma determinada tarefa.

Você seguirá o fluxo de trabalho geral de aprendizado de máquina:
1.   Examinar e entender os dados
2.   Crie um pipeline de entrada, neste caso usando Keras ImageDataGenerator
3.   Definir o modelo 
4.   Treine o modelo
5.   Avaliar modelo

In [ ]:
#@title Importes principais de bibliotecas

import matplotlib.pyplot as plt
import numpy as np
import os
--------------- TODO


# Pré-processamento de dados

Faça download e extraia um arquivo zip contendo as imagens e crie um `f.data.Dataset` para treinamento e validação usando o utilitário `tf.keras.utils.image_dataset_from_directory`

In [ ]:
SEED = 12

!wget --no-check-certificate \
    'https://www.dropbox.com/s/1wjwczpfddbbnb2/cat_and_dogs.zip' \
    -O "/tmp/cats-and-dogs.zip"

!unzip /tmp/cats-and-dogs.zip

In [ ]:
train_dir = 'training_set/training_set'
validation_dir = 'training_set/training_set'
test_dir = 'test_set/test_set'

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

# Imagens para treino do modelo
# Importante notar que estamos usando 85% dos dados para o treino
# e os dados restantes, ou seja, 15% para validação
--------------- TODO
(train_dir,
                                                            seed=SEED,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            validation_split=0.15,
                                                            subset='training')

In [ ]:
# Imagens para validação do modelo
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 seed=SEED,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE,
                                                                 validation_split=0.15,
                                                                 subset='validation')

In [ ]:
# Imagens para teste do modelo
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE,
                                                                 seed=SEED)

In [ ]:
#@title Visualizando algumas amostras do dataset

class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Configurar o conjunto de dados para desempenho

Use a pré-busca em buffer para carregar imagens do disco sem que a E/S se torne um bloqueio.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Usar o aumento de dados

Quando você não tem um grande conjunto de dados de imagem, é uma boa prática introduzir artificialmente a diversidade de amostra aplicando transformações aleatórias, porém realistas, às imagens de treinamento, como rotação e inversão horizontal.

In [ ]:
--------------- TODO

In [ ]:
#@title Vamos aplicar repetidamente essas camadas na mesma imagem e ver o resultado.
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Redimensionar valores de pixel

Em instantes, você fará o download do `tf.keras.applications.MobileNetV` para uso como modelo base. Este modelo espera valores de pixel em `[-1, 1]` , mas neste ponto, os valores de pixel em suas imagens estão em `[0, 255]`. Para redimensioná-los, use o método de pré-processamento incluído no modelo.

In [ ]:
--------------- TODO

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Rescaling
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

## Crie o modelo base a partir dos convnets pré-treinados

Você criará o modelo base a partir do modelo MobileNet V2 desenvolvido no Google. Isso é pré-treinado no conjunto de dados ImageNet, um grande conjunto de dados que consiste em `1,4 milhão de imagens e 1.000 classes`. 

Os recursos da última camada retêm mais generalidade. Primeiro, instancie um modelo MobileNet V2 pré-carregado com pesos treinados no ImageNet. Ao especificar o argumento `include_top=False`, você carrega uma rede que não inclui as camadas de classificação na parte superior, o que é ideal para extração de recursos.

In [ ]:
# Cria o modelo base a partir do modelo pré-treinado MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(
    input_shape = IMG_SHAPE,
    include_top = False,
    weights='imagenet'
)
--------------- TODO

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)

## Extração de recursos

Nesta etapa, você congelará a base convolucional criada na etapa anterior e usará como extrator de recursos. Além disso, você adiciona um classificador em cima dele e treina o classificador de nível superior.

In [ ]:
base_model.trainable = False

In [ ]:
# Vamos dar uma olhada na arquitetura do modelo base
base_model.summary()

### Adicionar um cabeçalho de classificação

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Aplique uma camada `tf.keras.layers.Dense` para converter esses recursos em uma única previsão por imagem. Você não precisa de uma função de ativação aqui porque essa previsão será tratada como um logit ou um valor bruto de previsão. Números positivos predizem a classe 1, números negativos predizem a classe 0.

In [ ]:
--------------- TODO

Construa um modelo encadeando as camadas de aumento de dados, redimensionamento, `base_model` e extrator de recursos usando a API funcional do Keras. Como mencionado anteriormente, use `training=False` pois nosso modelo contém uma camada `BatchNormalization` (será executada no modo de inferência e não atualizará suas estatísticas de média e variação).

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))

x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### Compilar o modelo

Compile o modelo antes de treiná-lo. Como existem duas classes, use a perda `tf.keras.losses.BinaryCrossentropy` com `from_logits=True`, pois o modelo fornece uma saída linear.

In [ ]:
base_learning_rate = 0.0001

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

### Treine o modelo

Após treinar por 10 épocas, você deverá ver ~ 94% de precisão no conjunto de validação.


In [ ]:
initial_epochs = 10

In [ ]:
--------------- TODO

In [ ]:
#@title Curvas de aprendizado

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Afinação

No experimento de extração de recursos, você estava treinando apenas algumas camadas em cima de um modelo base MobileNetV2. Os pesos da rede pré-treinada não foram atualizados durante o treinamento. 

Tudo o que você precisa fazer é descongelar o base_model e definir as camadas inferiores como não treináveis. Em seguida, você deve recompilar o modelo (necessário para que essas alterações tenham efeito) e retomar o treinamento.

In [ ]:
base_model.trainable = True

In [ ]:
# Vamos dar uma olhada para ver quantas camadas existem no modelo base
print("Number of layers in the base model: ", len(base_model.layers))

# Ajuste fino desta camada em diante
fine_tune_at = 100

# Congele todas as camadas antes da camada `fine_tune_at`
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Compilar o modelo

Como você está treinando um modelo muito maior e deseja readaptar os pesos pré-treinados, é importante usar uma taxa de aprendizado menor neste estágio.

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Continue treinando o modelo

Se você treinou para convergência anteriormente, esta etapa melhorará sua precisão em alguns pontos percentuais.

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Vamos dar uma olhada nas curvas de aprendizado da precisão/perda de treinamento e validação ao ajustar as últimas camadas do modelo base do MobileNetV2 e treinar o classificador em cima dele. A perda de validação é muito maior do que a perda de treinamento, então você pode ter algum overfitting.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Avaliação e previsão

In [ ]:
--------------- TODO

E agora você está pronto para usar esse modelo para prever se seu animal de estimação é um gato ou um cachorro.

In [ ]:
# Recupera um lote de imagens do conjunto de teste
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Aplica uma função sigmoide, pois nosso modelo retorna dados em logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")